#### We are trying to modify the power plants of this grid to obtain this energy mix:
<img src="images/target_em.png" width="30%" height="30%">

In [ ]:
import numpy as np
import os
import pandas as pd
import grid2op
from grid2op.Chronics import ChangeNothing
from chronix2grid.kpi.Generator_parameter_checker import EnergyMix_AprioriChecker
import plotly
import plotly.graph_objects as go
from grid2op.PlotGrid import NUKE_COLOR, THERMAL_COLOR, WIND_COLOR, SOLAR_COLOR, HYDRO_COLOR

# for pandas interactive plots
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)


#### We load the original prods_charac.csv

In [ ]:
env_name = "case118_l2rpn_wcci_benjamin"
path_ref = os.path.join("..", "example", "input", "generation")
path_tmp = os.path.join("..", "example", "custom", "input", "generation")
input_path = os.path.join(path_ref, env_name)
df = pd.read_csv(os.path.join(input_path, "prods_charac.csv"))

avg_pmaxs = df.groupby(["type"])["Pmax"].mean()
types = avg_pmaxs.index.to_numpy()
avg_pmaxs = avg_pmaxs.to_numpy()

In [ ]:
df

In [ ]:
df["type"].value_counts()

In [ ]:
def get_info_by_type(info):
  res = []
  for t in types:
    res.append(df[df["type"] == t].iloc[0][info])
  return np.array(res)

#### We recover some informations about each power plant type

In [ ]:
df = pd.read_csv(os.path.join(input_path, "prods_charac.csv"))


n = df.shape[0]
pmaxs = [250.0, 400.0, 74.7, 200.0, 67.2]
max_ramp_up = get_info_by_type("max_ramp_up")
max_ramp_down = get_info_by_type("max_ramp_down")
min_up_time = get_info_by_type("min_up_time")
min_down_time = get_info_by_type("min_down_time")
marginal_cost = get_info_by_type("marginal_cost")
shut_down_cost = get_info_by_type("shut_down_cost")
start_cost = get_info_by_type("start_cost")

In [ ]:
df["type"].value_counts()

#### We load the solution returned by the solver and modify the original csv according to the solution

YOUR "SOLVER" DO NOT WORK AT ALL !

In [ ]:
# file = open("optimization/solver/build/result.txt")
# for i in range(n):
#   idx = int(file.readline())
#   df.at[i, "type"] = types[idx]
#   df.at[i, "Pmax"] = pmaxs[idx]
#   df.at[i, "max_ramp_up"] = max_ramp_up[idx]
#   df.at[i, "max_ramp_down"] = max_ramp_down[idx]
#   df.at[i, "min_up_time"] = min_up_time[idx]
#   df.at[i, "min_down_time"] = min_down_time[idx]
#   df.at[i, "marginal_cost"] = marginal_cost[idx]
#   df.at[i, "shut_down_cost"] = shut_down_cost[idx]
#   df.at[i, "start_cost"] = start_cost[idx]

In [ ]:
# df

#### There is much more wind power plants

In [ ]:
df["type"].value_counts()

In [ ]:
output_path = os.path.join(path_tmp, env_name)
df.to_csv(os.path.join(output_path, "prods_charac.csv"), index=False)

In [ ]:
capacity_factor = np.array([30, 95, 15, np.nan, 25])
average_load = 2800
capacity_factor_df = pd.DataFrame(data=capacity_factor, columns=['capacity_factor'], index=types)

grid_path = os.path.join(output_path, "grid.json")
env118_withoutchron = grid2op.make(
    output_path,
    test=True,
    grid_path=grid_path, # assign it the 118 grid
    chronics_class=ChangeNothing, # tell it to change nothing (not the most usable environment...)
)

Target_EM_percentage=pd.DataFrame(data=[9, 36, 17, 2, 36], columns=['target_energy_mix'], index=types)

#Variable used to anticipate the energy mix a priori. Update them after chronics generation if too different
PeakLoad = 4200 #expected peak load

EnergyMix_AprioriChecker(env118_withoutchron, Target_EM_percentage, PeakLoad, average_load, capacity_factor_df)

#### You can see that the difference between the target energy mix and the apriori energy mix is 45%

# A) We now try to generate the loads / renewable to make sure the generated data are compatible with the grid

In [ ]:
### CONSTANT

notebook_folder=%pwd

# define your input folder
INPUT_FOLDER = os.path.join(notebook_folder, '..', 'example', 'custom', 'input')

OUTPUT_FOLDER = os.path.join(notebook_folder, '..', 'example', 'custom', 'output')

# Detailed configuration to be set in <INPUT_FOLDER>/<CASE>/params.json
weeks = 4
n_scenarios = 1
by_n_weeks = 4

mode = 'RLTK'
mode = 'RL'


CASE = env_name

load_seed = 7
renewable_seed = 12

li_months = ["2012-01-01", "2012-02-01", "2012-03-01", "2012-04-01", "2012-05-01", "2012-06-01",
             "2012-07-01", "2012-08-01", "2012-09-01", "2012-10-01", "2012-11-01", "2012-12-01",
            ]

In [ ]:
for start_date in li_months:
    cli_chronix2grid = "chronix2grid \
                        --mode {} --output-folder {} --input-folder {} --ignore-warnings \
                        --weeks {} --case {} --n_scenarios {} --start-date {} --by-n-weeks {} \
                        --seed-for-loads {}  --seed-for-res {}".format(
                        mode, OUTPUT_FOLDER, INPUT_FOLDER, weeks, CASE, n_scenarios, start_date,by_n_weeks,
                        load_seed, renewable_seed)
    cli_chronix2grid
    !$cli_chronix2grid

#### Load all data

In [ ]:
loads_p = []
prods_p_renewable = []
for start_date in li_months:
    path_data_generated = os.path.join(OUTPUT_FOLDER, "generation", CASE, start_date, "Scenario_0")
    loads_p.append(pd.read_csv(os.path.join(path_data_generated, "load_p.csv.bz2"), sep=";"))
    prods_p_renewable.append(pd.read_csv(os.path.join(path_data_generated, "prod_p.csv.bz2"), sep=";"))
loads_p = pd.concat(loads_p, ignore_index=True)
prods_p_renewable = pd.concat(prods_p_renewable, ignore_index=True)


In [ ]:
is_gen_solar = env118_withoutchron.gen_type == "solar"
is_gen_wind = env118_withoutchron.gen_type == "wind"
gen_solar_name = env118_withoutchron.name_gen[is_gen_solar]
gen_wind_name = env118_withoutchron.name_gen[is_gen_wind]
list_gen_renwable = gen_solar_name.tolist()+gen_wind_name.tolist()

In [ ]:
residual_load = loads_p.sum(axis=1) - prods_p_renewable[list_gen_renwable].sum(axis=1)
proportion_solar_wind = pd.DataFrame({"total_load": loads_p.sum(axis=1),
                                      "total_solar": prods_p_renewable[gen_solar_name].sum(axis=1),
                                      "total_wind": prods_p_renewable[gen_wind_name].sum(axis=1),
                                     })

In [ ]:
min_gen_possible = np.sum(env118_withoutchron.gen_pmin[env118_withoutchron.gen_redispatchable])
max_gen_possible = np.sum(env118_withoutchron.gen_pmax[env118_withoutchron.gen_redispatchable])
max_gen_up_possible = np.sum(env118_withoutchron.gen_max_ramp_up[env118_withoutchron.gen_redispatchable])
max_gen_down_possible = np.sum(env118_withoutchron.gen_max_ramp_down[env118_withoutchron.gen_redispatchable])
loss_ratio = 0.95  # takes into account some loss (conservative)

#### Some plot to have a look at the generation, might be handy

In [ ]:
prods_p_renewable[gen_wind_name[1]].iplot()

In [ ]:
prods_p_renewable[gen_solar_name[1]].iplot()

we can compare above with previous validated grid2op environment (uncomment)

In [ ]:
# import grid2op
# env_test = grid2op.make("l2rpn_neurips_2020_track2_small")
# this_env_wind = [i for i in range(env_test.n_gen) if env_test.gen_type[i] == "wind"]

In [ ]:
# pd.DataFrame(env_test.chronics_handler.real_data.data.prod_p[:, this_env_wind[3]]).iplot()

### Expected share of renewables

If everything goes well and there is not much "curtailment"

In [ ]:
labels = ['solar','wind','unknown']
values = [proportion_solar_wind["total_solar"].sum(),
          proportion_solar_wind["total_wind"].sum(),
          proportion_solar_wind["total_load"].sum() - proportion_solar_wind["total_solar"].sum() - proportion_solar_wind["total_wind"].sum()
         ]

fig = go.Figure(data=[go.Pie(labels=labels,
                             values=values,
                             marker_colors=[SOLAR_COLOR, WIND_COLOR, 'rgba(0,0,0, 0.05)'],
                             text=[f"{round(el / 12., -3):,.0f} MWh" for el in values])
                     ]
               )
fig.update_layout(
    title=f"This has only run on {weeks} weeks, the target is for a whole year !!!"
)
fig.show()


### Basic infeasibilities

The section bellow check that the market dispatch has a chance to converge. If something is violated here, 
then there is now way the market dispatch can work

In [ ]:
due_pmin = residual_load < min_gen_possible
due_pmax = residual_load > loss_ratio * max_gen_possible
delta_gen = residual_load.diff()  # diff(t) = residual_load(t) - residual_load(t-1)
due_rampmax = delta_gen > max_gen_up_possible
due_rampmin = delta_gen < -max_gen_up_possible
due_rampmax[::(weeks * 7 * 288)] = False  # remove the "interface" at 4 weeks, 8 weeks etc.
due_rampmin[::(weeks * 7 * 288)] = False  # remove the "interface" at 4 weeks, 8 weeks etc.
print(f"There are (at least) {due_pmin.sum()} infeasibilities due to pmin ({100. * due_pmin.sum() / due_pmin.shape[0]:.0f}%) that will require curtailment")
print(f"There are (at least) {due_pmax.sum()} infeasibilities due to pmax ({100. * due_pmax.sum() / due_pmax.shape[0]:.0f}%) that will require curtailment")
print(f"There are (at least) {due_rampmax.sum()} infeasibilities due to ramp up ({100. * due_rampmax.sum() / due_rampmax.shape[0]:.0f}%) that will require curtailment")
print(f"There are (at least) {due_rampmin.sum()} infeasibilities due to ramp down ({100. * due_rampmin.sum() / due_rampmin.shape[0]:.0f}%) that will require curtailment")
# assert np.all(~due_pmin) and np.all(~due_pmax) and np.all(~due_rampmax) and np.all(~due_rampmin)

In [ ]:
delta_gen.max()

In [ ]:
max_gen_up_possible

### some basic check to make sure the data are consistent with what we asked

In [ ]:
all_solar_above_pmax = prods_p_renewable[gen_solar_name] > env118_withoutchron.gen_pmax[is_gen_solar]
all_solar_below_pmin = prods_p_renewable[gen_solar_name] < env118_withoutchron.gen_pmin[is_gen_solar]

all_wind_above_pmax = prods_p_renewable[gen_wind_name] > env118_withoutchron.gen_pmax[is_gen_wind]
all_wind_below_pmin = prods_p_renewable[gen_wind_name] < env118_withoutchron.gen_pmin[is_gen_wind]

assert np.all(all_solar_above_pmax.sum() == 0), f"some solar are above pmax:\n{all_solar_above_pmax.sum()}"
assert np.all(all_solar_below_pmin.sum() == 0), f"some solar are below pmin:\n{all_solar_below_pmin.sum()}"
assert np.all(all_wind_above_pmax.sum() == 0), f"some wind are above pmax:\n{all_wind_above_pmax.sum()})"
assert np.all(all_wind_below_pmin.sum() == 0), f"some wind are below pmin:\n{all_wind_below_pmin.sum()}"

# B) Now start the redispatching

In [ ]:
MODE_DISPATCH2 = "RLDT"
seed_dispatch = 0

In [ ]:
%%time
for start_date in li_months:
    cli_chronix2grid2 = "chronix2grid \
                         --mode {} --output-folder {} --input-folder {} --ignore-warnings \
                         --weeks {} --case {} --n_scenarios {} --start-date {} --by-n-weeks {} \
                         --seed-for-loads {}  --seed-for-res {} --seed-for-dispatch {}".format(
                         MODE_DISPATCH2, OUTPUT_FOLDER, INPUT_FOLDER, weeks, CASE, n_scenarios, start_date,by_n_weeks,
                         load_seed, renewable_seed, seed_dispatch)
    cli_chronix2grid2
    !$cli_chronix2grid2

In [ ]:
loads_p2 = []
prods_p_total = []
prods_p_total_gen = []
for start_date in li_months:
    path_data_generated = os.path.join(OUTPUT_FOLDER, "generation", CASE, start_date, "Scenario_0")
    loads_p2.append(pd.read_csv(os.path.join(path_data_generated, "load_p.csv.bz2"), sep=";"))
    prods_p_total.append(pd.read_csv(os.path.join(path_data_generated, "prod_p.csv.bz2"), sep=";"))
    prods_p_total_gen.append(pd.read_csv(os.path.join(path_data_generated, "prod_p_renew_orig.csv.bz2"), sep=";"))
    
loads_p2 = pd.concat(loads_p2, ignore_index=True)
prods_p_total = pd.concat(prods_p_total, ignore_index=True)
prods_p_total_gen = pd.concat(prods_p_total_gen, ignore_index=True)


In [ ]:
gen_solar_name2 = env118_withoutchron.name_gen[env118_withoutchron.gen_type == "solar"]
gen_wind_name2 = env118_withoutchron.name_gen[env118_withoutchron.gen_type == "wind"]
gen_hydro_name2 = env118_withoutchron.name_gen[env118_withoutchron.gen_type == "hydro"]
gen_nuclear_name2 = env118_withoutchron.name_gen[env118_withoutchron.gen_type == "nuclear"]
gen_thermal_name2 = env118_withoutchron.name_gen[env118_withoutchron.gen_type == "thermal"]
residual_load = loads_p2.sum(axis=1) - prods_p_total.sum(axis=1)
proportion_solar_wind2 = pd.DataFrame({"total_load": loads_p2.sum(axis=1),
                                      "total_solar": prods_p_total[gen_solar_name2].sum(axis=1),
                                      "total_wind": prods_p_total[gen_wind_name2].sum(axis=1),
                                      "total_hydro": prods_p_total[gen_hydro_name2].sum(axis=1),
                                      "total_nuclear": prods_p_total[gen_nuclear_name2].sum(axis=1),
                                      "total_thermal": prods_p_total[gen_thermal_name2].sum(axis=1),
                                     })

In [ ]:
labels2 = ['solar','wind','hydro', "nuclear", "thermal"]
values2 = [proportion_solar_wind2["total_solar"].sum(),
           proportion_solar_wind2["total_wind"].sum(),
           proportion_solar_wind2["total_hydro"].sum(),
           proportion_solar_wind2["total_nuclear"].sum(),
           proportion_solar_wind2["total_thermal"].sum(),
          ]

fig = go.Figure(data=[go.Pie(labels=labels2,
                             values=values2,
                             marker_colors=[SOLAR_COLOR, WIND_COLOR, HYDRO_COLOR, NUKE_COLOR, THERMAL_COLOR],
                            text=[f"{round(el / 12., -3):,.0f} MWh" for el in values2]
                            )]
                             
                )
fig.update_layout(
    title=f"This has only run on {weeks} weeks, the target is for a whole year !!!"
)
fig.show()

### Check the "OPF" did not curtail too much

Be carefull, there are "noise" in the generation now ! This is why the "prod_p_renew_orig" has been loaded and is used here !

In [ ]:
# due to rounding
assert np.all((prods_p_total[gen_wind_name] - prods_p_total_gen[gen_wind_name2]).max() <= 0.100001)

In [ ]:
# max amount "curtailed" by the optimiser, we should get close to 0.
(prods_p_total[gen_wind_name] - prods_p_total_gen[gen_wind_name2]).min()

In [ ]:
# all of that should be really, really close to 0., it's the amount
# of energy "lost" because of constraints on the controlable generators
total_curtailed_generation_pct = 100. * (prods_p_total[gen_wind_name] - prods_p_total_gen[gen_wind_name2]).abs().sum() / prods_p_total_gen[gen_wind_name].sum()
total_curtailed_generation_pct

In [ ]:
#prods_p_total_gen[gen_wind_name2[0]].iplot()
nm_gen = gen_wind_name[0]
fig = go.Figure(data=[go.Scatter(y=prods_p_total[nm_gen], name="actual generation"),
                      go.Scatter(y=prods_p_total_gen[nm_gen], name="possible generation"),
                     ]
               )
fig.update_layout(
    title=f"Comparison of generation for wind generator {nm_gen}"
)
fig.show()

In [ ]:
# due to rounding
assert np.all((prods_p_total[gen_solar_name] - prods_p_total_gen[gen_solar_name2]).max() <= 0.100001)

In [ ]:
# max amount "curtailed" by the optimiser, we should get close to 0.
(prods_p_total[gen_solar_name] - prods_p_total_gen[gen_solar_name2]).min()

In [ ]:
# all of that should be really, really close to 0., it's the amount
# of energy "lost" because of constraints on the controlable generators
total_curtailed_generation_pct = 100. * (prods_p_total[gen_solar_name] - prods_p_total_gen[gen_solar_name2]).abs().sum() / prods_p_total_gen[gen_solar_name].sum()
total_curtailed_generation_pct

In [ ]:
nm_gen = gen_solar_name[0]
fig = go.Figure(data=[go.Scatter(y=prods_p_total[nm_gen], name="actual generation"),
                      go.Scatter(y=prods_p_total_gen[nm_gen], name="possible generation"),
                     ]
               )
fig.update_layout(
    title=f"Comparison of generation for solar generator {nm_gen}"
)
fig.show()

### Check possible bugs

#### check loads meet demands

In [ ]:
ratio_loss = prods_p_total.sum(axis=1) / loads_p.sum(axis=1)
assert (ratio_loss.max() - ratio_loss.min()) <= 3e-3  # with rounding this can vary a little

#### for renewable generators

In [ ]:
all_solar_above_pmax = prods_p_total[gen_solar_name] > env118_withoutchron.gen_pmax[is_gen_solar]
all_solar_below_pmin = prods_p_total[gen_solar_name] < env118_withoutchron.gen_pmin[is_gen_solar]

all_wind_above_pmax = prods_p_total[gen_wind_name] > env118_withoutchron.gen_pmax[is_gen_wind]
all_wind_below_pmin = prods_p_total[gen_wind_name] < env118_withoutchron.gen_pmin[is_gen_wind]

assert np.all(all_solar_above_pmax.sum() == 0), f"some solar are above pmax:\n{all_solar_above_pmax.sum()}"
assert np.all(all_solar_below_pmin.sum() == 0), f"some solar are below pmin:\n{all_solar_below_pmin.sum()}"
assert np.all(all_wind_above_pmax.sum() == 0), f"some wind are above pmax:\n{all_wind_above_pmax.sum()})"
assert np.all(all_wind_below_pmin.sum() == 0), f"some wind are below pmin:\n{all_wind_below_pmin.sum()}"


#### for the others

In [ ]:
def check_controlable_gens(gen_name, env, prods_p_total):
    pmax_ = np.array([env.gen_pmax[np.where(env.name_gen == nm_)[0]] for nm_ in gen_name])
    pmax_ = pmax_.ravel()
    check_pmax = prods_p_total[gen_name] > pmax_

    pmin_ = np.array([env.gen_pmin[np.where(env.name_gen == nm_)[0]] for nm_ in gen_name])
    pmin_ = pmin_.ravel()
    check_pmin = prods_p_total[gen_name] < pmin_

    max_up_ = np.array([env.gen_max_ramp_up[np.where(env.name_gen == nm_)[0]] for nm_ in gen_name])
    max_up_ = max_up_.ravel()
    max_down_ = np.array([env.gen_max_ramp_down[np.where(env.name_gen == nm_)[0]] for nm_ in gen_name])
    max_down_ = max_down_.ravel()
    delta_gen_ = prods_p_total[gen_name].diff()  # prods_p_total.diff(t) = prods_p_total(t) - prods_p_total(t-1)
    check_max_up = delta_gen_ > max_up_
    check_max_down = delta_gen_ < -max_down_
    # remove the "interface" between the months
    check_max_up[::(weeks * 7 * 288 - 1)] = False
    check_max_down[::(weeks * 7 * 288 - 1)] = False
    check_max_up[::(weeks * 7 * 288 )] = False
    check_max_down[::(weeks * 7 * 288)] = False
    return check_pmax, check_pmin, check_max_up, check_max_down

def check_all_controlable_gens(prods_p_total, gen_hydro_name2, gen_nuclear_name2, gen_thermal_name2, env118_withoutchron):
    check_pmax_hydro, check_pmin_hydro, check_max_up_hydro, check_max_down_hydro = check_controlable_gens(
        gen_hydro_name2, env118_withoutchron, prods_p_total)
    check_pmax_nuclear, check_pmin_nuclear, check_max_up_nuclear, check_max_down_nuclear = check_controlable_gens(
        gen_nuclear_name2, env118_withoutchron, prods_p_total)
    check_pmax_thermal, check_pmin_thermal, check_max_up_thermal, check_max_down_thermal = check_controlable_gens(
        gen_thermal_name2, env118_withoutchron, prods_p_total)

    assert np.all(check_pmax_hydro.sum() == 0), f"some hydro are above pmax:\n{all_solar_above_pmax.sum()}"
    assert np.all(check_pmin_hydro.sum() == 0), f"some hydro are below pmin:\n{all_solar_below_pmin.sum()}"
    assert np.all(check_max_up_hydro.sum() == 0), f"some hydro are above max_up:\n{check_max_up_hydro.sum()}"
    assert np.all(check_max_down_hydro.sum() == 0), f"some hydro are below max_down:\n{check_max_down_hydro.sum()}"

    assert np.all(check_pmax_nuclear.sum() == 0), f"some nuclear are above pmax:\n{check_pmax_nuclear.sum()}"
    assert np.all(check_pmin_nuclear.sum() == 0), f"some nuclear are below pmin:\n{check_pmin_nuclear.sum()}"
    assert np.all(check_max_up_nuclear.sum() == 0), f"some nuclear are above max_up:\n{check_max_up_nuclear.sum()}"
    assert np.all(check_max_down_nuclear.sum() == 0), f"some nuclear are below max_down:\n{check_max_down_nuclear.sum()}"

    assert np.all(check_pmax_thermal.sum() == 0), f"some thermal are above pmax:\n{check_pmax_thermal.sum()}"
    assert np.all(check_pmin_thermal.sum() == 0), f"some thermal are below pmin:\n{check_pmin_thermal.sum()}"
    assert np.all(check_max_up_thermal.sum() == 0), f"some thermal are above max_up:\n{check_max_up_thermal.sum()}"
    assert np.all(check_max_down_thermal.sum() == 0), f"some thermal are below max_down:\n{check_max_down_thermal.sum()}"

    print("All checks passed !\n")

check_all_controlable_gens(prods_p_total, gen_hydro_name2, gen_nuclear_name2, gen_thermal_name2, env118_withoutchron)
print(f"It's now time to look at the 'KPI' and the productions / loads generated at :\n\t{path_data_generated}")

# C) Check the loss of the grid

Make sure the data generated can be loaded with grid2op framework

This part might not work on microsoft windows...

In [ ]:
from grid2op.Parameters import Parameters
from lightsim2grid import LightSimBackend  # might need "pip install lightsim2grid"
import shutil
from grid2op.Chronics import FromNPY
# TODO TQDM !

path_chronics_outputopf = os.path.join(OUTPUT_FOLDER, "all_scenarios")
shutil.rmtree(path_chronics_outputopf)
if not os.path.exists(path_chronics_outputopf):
    os.mkdir(path_chronics_outputopf)
    
path_chronics_fixed = os.path.join(OUTPUT_FOLDER, "fixed_chronics")

In [ ]:
for start_date in li_months:
    path_data_generated = os.path.join(OUTPUT_FOLDER, "generation", CASE, start_date, "Scenario_0")
    path_ = os.path.abspath(os.path.join(path_chronics_outputopf, start_date))
    os.symlink(path_data_generated, path_)

In [ ]:
param = env118_withoutchron.parameters
param.NO_OVERFLOW_DISCONNECTION = True
env_for_loss = grid2op.make(
    output_path,
    test=True,
    grid_path=grid_path, # assign it the 118 grid
    chronics_path=path_chronics_outputopf,
    param=param,
    backend=LightSimBackend()
    )

In [ ]:
def fill_real_gen(target, row_id, obs, env):
    target[row_id, env.gen_renewable] = obs.gen_p[env.gen_renewable]
    loss = np.sum(obs.gen_p) - np.sum(obs.load_p)  # actual loss of the grid
    
    # split what the slack absorbed in the controlable generators
    gen_p_setpoint = env.chronics_handler.real_data.data.prod_p[row_id]
    to_split = np.sum(obs.gen_p) - np.sum(gen_p_setpoint)
    redisp_ = 1.0 * gen_p_setpoint[~env.gen_renewable]
    redisp_ *= 1.0 + to_split / np.sum(redisp_)
    target[row_id, ~env.gen_renewable] = redisp_
    return loss

In [ ]:
final_gen_p = np.full((weeks * 7 * 288 - 1, env_for_loss.n_gen), fill_value=np.NaN, dtype=np.float32)
final_gen_v = np.full((weeks * 7 * 288 - 1, env_for_loss.n_gen), fill_value=np.NaN, dtype=np.float32)
final_load_p = np.full((weeks * 7 * 288 - 1, env_for_loss.n_load), fill_value=np.NaN, dtype=np.float32)
final_load_q = np.full((weeks * 7 * 288 - 1, env_for_loss.n_load), fill_value=np.NaN, dtype=np.float32)
all_loss = np.zeros(weeks * 7 * 288 - 1)

obs = env_for_loss.reset()
i = 0
all_loss[i] = fill_real_gen(final_gen_p, i, obs, env_for_loss)
final_gen_v[i] = obs.gen_v
final_load_p[i] = obs.load_p
final_load_q[i] = obs.load_q
chron_name = env_for_loss.chronics_handler.get_id()
done = False
while not done:
    obs, reward, done, info = env_for_loss.step(env_for_loss.action_space())
    i += 1
    all_loss[i] = fill_real_gen(final_gen_p, i, obs, env_for_loss)
    final_gen_v[i] = obs.gen_v
    final_load_p[i] = obs.load_p
    final_load_q[i] = obs.load_q

In [ ]:
# these numbers should be between 1-2-3% (more than 5 indicates an issue !)
losses_pct = 100. * all_loss / np.sum(final_gen_p, axis=1)
print(f"max loss: {losses_pct.max():.2f} %")
print(f"min loss: {losses_pct.min():.2f} %")
print(f"avg loss: {losses_pct.mean():.2f} %")
assert losses_pct.max() <= 5

In [ ]:
# test that it's working (and continue looping until it does not move)
env_fixed = grid2op.make(
    output_path,
    test=True,
    grid_path=grid_path, # assign it the 118 grid
    param=param,
    backend=LightSimBackend(),
    chronics_class=FromNPY,
    chronics_path=path_chronics_outputopf,
    data_feeding_kwargs={"load_p": final_load_p,
                         "load_q": final_load_q,
                         "prod_p": final_gen_p,
                         "prod_v": final_gen_v}
    )

In [ ]:
# this should be as close to 0. as possible...
# we might do a second "repartition loop" to make sure it's ok :-)
i = 0
final_gen_p2 = final_gen_p * np.NaN
diff_ = np.full((weeks * 7 * 288 - 1, env_fixed.n_gen), fill_value=np.NaN)
obs = env_fixed.reset()
diff_[i] = obs.gen_p - final_gen_p[i]
fill_real_gen(final_gen_p2, i, obs, env_for_loss)
while True:
    obs, reward, done, info = env_fixed.step(env_for_loss.action_space())
    if done:
        break
    i += 1
    fill_real_gen(final_gen_p2, i, obs, env_for_loss)
    diff_[i] = obs.gen_p - final_gen_p[i]

In [ ]:
np.abs(diff_).max()

In [ ]:
# TODO use the time serie module from lightsim2grid for that !!!
env_fixed2 = grid2op.make(
    output_path,
    test=True,
    grid_path=grid_path, # assign it the 118 grid
    param=param,
    backend=LightSimBackend(),
    chronics_class=FromNPY,
    chronics_path=path_chronics_outputopf,
    data_feeding_kwargs={"load_p": final_load_p,
                         "load_q": final_load_q,
                         "prod_p": final_gen_p2,
                         "prod_v": final_gen_v}
    )
i = 0
final_gen_p3 = final_gen_p * np.NaN
diff_2 = np.full((weeks * 7 * 288 - 1, env_fixed.n_gen), fill_value=np.NaN)
obs = env_fixed2.reset()
diff_2[i] = obs.gen_p - final_gen_p2[i]
fill_real_gen(final_gen_p3, i, obs, env_for_loss)
while True:
    obs, reward, done, info = env_fixed2.step(env_for_loss.action_space())
    if done:
        break
    i += 1
    fill_real_gen(final_gen_p3, i, obs, env_for_loss)
    diff_2[i] = obs.gen_p - final_gen_p2[i]

In [ ]:
np.abs(diff_2).max()

In [ ]:
env_fixed3 = grid2op.make(
    output_path,
    test=True,
    grid_path=grid_path, # assign it the 118 grid
    param=param,
    backend=LightSimBackend(),
    chronics_class=FromNPY,
    chronics_path=path_chronics_outputopf,
    data_feeding_kwargs={"load_p": final_load_p,
                         "load_q": final_load_q,
                         "prod_p": final_gen_p3,
                         "prod_v": final_gen_v}
    )
i = 0
final_gen_p4 = final_gen_p * np.NaN
diff_3 = np.full((weeks * 7 * 288 - 1, env_fixed.n_gen), fill_value=np.NaN)
obs = env_fixed3.reset()
diff_3[i] = obs.gen_p - final_gen_p3[i]
fill_real_gen(final_gen_p4, i, obs, env_for_loss)
while True:
    obs, reward, done, info = env_fixed3.step(env_for_loss.action_space())
    if done:
        break
    i += 1
    fill_real_gen(final_gen_p4, i, obs, env_for_loss)
    diff_3[i] = obs.gen_p - final_gen_p3[i]

In [ ]:
np.abs(diff_3).max()

now that the generation "does not move", we check that it meets the physical constraints

In [ ]:
final_gen_df = pd.DataFrame(final_gen_p3, columns=env_for_loss.name_gen)
check_all_controlable_gens(final_gen_df, gen_hydro_name2, gen_nuclear_name2, gen_thermal_name2, env118_withoutchron)

In [ ]:
np.abs(final_gen_df["gen_25_13"].diff()).max()

now we save the data in the right format

In [ ]:
# pd.DataFrame(final_load_p, columns=env_for_loss.name_load).to_csv(...)